In [1]:
# Add dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
from datetime import datetime
import requests
from config import weather_api_key
from citipy import citipy

In [2]:
# Starting url for weather api 
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [3]:
lats = np.random.uniform(low=-90.000, high=90.000, size=10)
lngs = np.random.uniform(low=-180.000, high=180.000, size=10)

In [4]:
# Use tuple() to display lat and lng combinations
coordinates = zip(lats, lngs)
for coordinate in coordinates:
    print(citipy.nearest_city(coordinate[0], coordinate[1]).city_name)
 

puerto ayora
ostrovnoy
the pas
jieshi
kisangani
carnarvon
verkhnetulomskiy
carnarvon
hithadhoo
jamestown


In [5]:
coordinates = zip(lats, lngs)
# Create list for holding cities
cities = []

# Identify nearest city for each lat and lng combination
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    # If the city is unique, add city to list
    if city not in cities:
        cities.append(city)
        
# Print the city count to confirm sufficient count
print(len(cities))
print(cities[0])

9
puerto ayora


In [12]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all cities in list
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

    # If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | puerto ayora
Processing Record 2 of Set 1 | ostrovnoy
Processing Record 3 of Set 1 | the pas
Processing Record 4 of Set 1 | jieshi
Processing Record 5 of Set 1 | kisangani
Processing Record 6 of Set 1 | carnarvon
Processing Record 7 of Set 1 | verkhnetulomskiy
Processing Record 8 of Set 1 | hithadhoo
Processing Record 9 of Set 1 | jamestown
-----------------------------
Data Retrieval Complete      
-----------------------------


In [9]:
len(city_data)

9

In [10]:
# Convert city_data to a data frame
city_data_df = pd.DataFrame(city_data)
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Puerto Ayora,-0.7393,-90.3518,78.76,77,13,9.53,EC,2022-04-28 22:17:30
1,Ostrovnoy,68.0531,39.5131,20.98,78,100,22.35,RU,2022-04-28 22:24:41
2,The Pas,53.8251,-101.2541,46.69,73,100,8.77,CA,2022-04-28 22:24:42
3,Jieshi,22.8134,115.8257,76.53,91,19,4.25,CN,2022-04-28 22:24:42
4,Kisangani,0.5167,25.2000,71.44,97,99,2.46,CD,2022-04-28 22:19:45


In [11]:
# Reorder the columns City, Country, Date, Lat, Lng, Max Temp, Humidity, Cloudiness, and Wind Speed
new_column_order = ["City", "Country", "Date", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed"]
city_data_df = city_data_df[new_column_order]
city_data_df.head()

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Puerto Ayora,EC,2022-04-28 22:17:30,-0.7393,-90.3518,78.76,77,13,9.53
1,Ostrovnoy,RU,2022-04-28 22:24:41,68.0531,39.5131,20.98,78,100,22.35
2,The Pas,CA,2022-04-28 22:24:42,53.8251,-101.2541,46.69,73,100,8.77
3,Jieshi,CN,2022-04-28 22:24:42,22.8134,115.8257,76.53,91,19,4.25
4,Kisangani,CD,2022-04-28 22:19:45,0.5167,25.2000,71.44,97,99,2.46


In [ ]:
# Create output file
output_data_file = "weather_data/cities.csv"

# Export data into CSV file
# city_data_df.to_csv(output_data_file, index_label="City_ID")